# How can we interact with a blockchain?

So, we have all these nodes distributed all over the world which are able to trustlessly save immutable transactions over a distributed ledger. Nice, but: how can we use it?
At first, we need an account for it. 

## Getting an address


### At first, let's use the "secrets" library for generating a new private key
Beware: for a lot of reasons which include the safety of your private key, NEVER use unsafe random key generators when you create a private key that you will use on a production blockchain. This can easily translate in a hacked account and in **all the money lost**.
This is related to the fact that random number generation performed by computers is not truly random, since the current computational devices are inherently deterministic. So, unless you have a quantum computer at home, it is better if you use safety specific random generators which include some external random input in the private key creation.
In this example we will use secrets library, which i suggest you to not use in a production environment.

Also, we will use the already known and installed package *eth_utils* to perform operations on the keys, together with *eth_keys* one. You can find instructions on how to install this package on <a href="https://github.com/ethereum/eth-keys">eth-keys page on GitHub</a>.

In [75]:
import secrets #a secure random hex generator for python
from eth_keys import keys
from eth_utils import decode_hex

#A working address with some ether on it
fixed_private = '5619099d74e5a0c24616de8eabbf4c3a2610d52caa8a7ea18e48ad3a895488f2' 
fixed_priv_key_bytes = decode_hex(fixed_private) #decode it in a bytes object
fixed_priv_key = keys.PrivateKey(fixed_priv_key_bytes) #convert it to hex representation
fixed_pub_key = fixed_priv_key.public_key #and calculate the public key
fixed_address = fixed_pub_key.to_checksum_address()

my_private = secrets.token_hex(32) #generate a random 32 digits hexadecimal, 
#this will be the bytes representation of our private key
print ('Private key:', my_private)
priv_key_bytes = decode_hex(my_private) #decode it in a bytes object
priv_key = keys.PrivateKey(priv_key_bytes) #convert it to hex representation
pub_key = priv_key.public_key #and calculate the public key
#Finally, extract our Ethereum address from the public key
address = pub_key.to_checksum_address()
print ("Address: ", address)

Private key: ab11ac7614e7c6e2d705478a8b4db7e8526416a5e0f92dea8a2e58c2d95a459a
Address:  0x1B4f9DF1Cb5bB802D53083317a91E56Fa1DeCc7c


## Connect to the chain

### Install the web3 APIs
In order to interact with the ethereum network, you will have to speak its language. To do so, you will have to use the web3 APIs, a library which "translates" code to ethereum commands.
Web3 APIs exist for a great variety of programming languages, but since we are wokring on python, we will have to install the python web3 APIs. If you did not do it already, you can always follow the instructions it the python web3 <a href="https://github.com/ethereum/web3.py">page on GitHub</a>.
Otherwise, you can just run the following command on shell:

### Select your Ethereum node
Now, we will have to connect to an existing ethereum chain. This can be done in two ways:
- **hosting a local node:** this means we will be come one of the mining nodes of the whole network, and we would be able to be part of the P2P network sustaining the platform. This implies downloading the full blockchain before having the possibility to do so, which will be infeasible for our purposes;
- **use a hosted remote node:** in principle you can use any ethereum node, provided you have access to it, **but remember: it is not a node controlled by you, so it can make a lot of bad things with your (virtual) money**.
Since we will test our system on a test ethereum network, for which the existing eth can be so easily obtained that have no value at all, we will use an external node hosted on <a href="https://infura.io">https://infura.io</a>.

In order to have access to an infura node, the only thing you just have to do is to register and create an account, and get a project url to which our node will connect.

In this case, I already did it for you, and you can access to a infura hosted node at this url: <a href="https://ropsten.infura.io/v3/53b3b1921ddf43f58cabf2eeb7ea7696">href="https://kovan.infura.io/v3/53b3b1921ddf43f58cabf2eeb7ea7696</a>. If something happens during your tests, however, you can easily register at the infura website and use the url that they will provide to you.

This particular node connects to the *ropsten* network, which is one of the ethereum test networks. This network runs **exactly** the same ethereum mainnet protocols and can be used for safely testing Ethereum Dapps, without risking precious eth.

Well, now we have a private address (which means, barely, an account), the APIs and a node. Let's start interacting with the blockchain!


### At first, let's get some money
the balace of your account can be checked by looking for its balance. The node will just follow the entire history of the blockchain, and compute the current account balance by (roughly) summing the previous transactions. 

Let's check if we can connect to the node:

In [69]:
from web3 import HTTPProvider, Web3, exceptions
w3 = Web3(HTTPProvider("https://ropsten.infura.io/v3/53b3b1921ddf43f58cabf2eeb7ea7696"))
print(w3.isConnected())

True


Then, let's check our balance.

In [77]:
myBalance = w3.eth.getBalance(address)
print (myBalance)

0


If everything went well, at this point you should have been able to connect to the ropsten network through infura, and to get the balance of your account (which, unless you are way extremely lucky, should be 0)

### Time to get some money
Since we are working on a test network (Ropsten), it is extremely easy to get money (or better, the Ethreum virtual currency, eth). This can be done by using a faucet, e.g. a website in ehich you give your address as input and you get eth on it in exchange.
To do so, go on the faucet <a href="https://faucet.ropsten.be">website</a>, input your address (Remember? You print it in the previous sections) and you will magically receive some test ethers.
at first, let'check if everything worked well:


In [36]:
myBalance = w3.eth.getBalance(address)
print ("Balance in gweis:", myBalance)
print ("Balance in eth:",  Web3.fromWei(myBalance, 'ether'))

Balance in gweis: 1000000000000000000
Balance in eth: 1


Great, we just got 1 eth from the faucet. With this sum, we should be able to perform all the tests we want.

## Sending ethers to other accounts
One of the first things we can do is to send money to other accounts.
This can be done, again , by using web3 API.

As any other transaction performed on the ethereum network, sending money to other accounts changes the internal state of memory on the blockchain, this means that miners should be paid for processing this information. 

The payment of these fees is managed internally by ethereum APIs, which associate a cost to each transaction. This cost is calculated in *gas*, a virtual quantity which roughly estimates the computational complexity of the required transaction. 

Since each transaction has a *gas* cost, miners can be paid by offering them a certain amount of *ether* per each unit of *gas* processed. 

### How to send a transaction
In order to send a transaction, we should send the corrects instructions to the ethereum network through our node. In order to prove that the transaction order came from the real owner of the address, it should be digitally signed by means of our private key (in the end, this is the reason because it should be kept private, since anyone which know it can digitally sign and commend any transaction).
Then, the digitally signed transaction should be send to the Eth network, where the nodes will verify it and perform its instructions, appending their effects on the upcoming block.
how to do it is shown in the next code block:

In [60]:
#Check the target address balance
target_address = '0xd3CdA913deB6f67967B99D67aCDFa1712C293601'
targetBalance = w3.eth.getBalance(target_address)
print ("Original balance in gweis:", targetBalance)
print ("Original balance in eth:",  Web3.fromWei(targetBalance, 'ether'))

#at first, define a eth transaction
my_transaction = {
    "nonce":w3.eth.getTransactionCount(address), #get the increasing id of the transaction
    "gasPrice":w3.eth.gasPrice, #set gas price to the current average
    "gas":100000, #Set the maximum gas that should be paid by the transaction
    "to":target_address, #Set the address to which teh transaction will be sent
    "value":12345, #Define the amount of gwei (one billionth of ether) that should be sent from our address to the new address
    "data":b'', #Yo can ìalways add a small message to teh transaction. You will pay for it.
}

#now, let's digitally sign the transaction
signed_txn = w3.eth.account.signTransaction(
    my_transaction, #the transaction dictionary
    priv_key, #as defined in the first code block
)

#And finally, send the transaction. (It returns the digital signature of the transaction)
transaction_signature = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
print (transaction_signature)

Original balance in gweis: 2196500005635325313
Original balance in eth: 2.196500005635325313
b"*\x99\xf48\x8f~\x95\x1a\xa3\\\\\xb2y!%&r\xaf\x96*\x0b>5\xbc\x9aC/\x8ec\x91V'"


In [63]:
#Just wait some time that the transaction gets accepted
import time
time.sleep(1)

#Print the transaction info
print (w3.eth.getTransaction(transaction_signature))

#check the new balance of the target address:
targetBalance = w3.eth.getBalance(target_address)
print ("Target balance in gweis:", targetBalance)
print ("target balance in eth:",  Web3.fromWei(targetBalance, 'ether'))

#And also check the new balance of our address
myBalance = w3.eth.getBalance(address)
print ("Balance in gweis:", myBalance)
print ("Balance in eth:",  Web3.fromWei(myBalance, 'ether'))

AttributeDict({'blockHash': HexBytes('0x918a47b9378ecdc02bb78880378a5ddc8ef80da49b4d0f5a443b3df670b098d4'), 'blockNumber': 7944892, 'from': '0x5DaD165F2bC49e95a969C104C9c887E3dAceDD3d', 'gas': 100000, 'gasPrice': 421875000, 'hash': HexBytes('0x2a99f4388f7e951aa35c5cb27921252672af962a0b3e35bc9a432f8e63915627'), 'input': '0x', 'nonce': 5, 'r': HexBytes('0x1993792de392c1b5d2341c9dd40db191091d2d0993cc4a8363db6eb1d7ad6e2b'), 's': HexBytes('0x6621632aaa03526eb6d4c2e4147c129bb90b7ca00cad49660509d477d34249e2'), 'to': '0xd3CdA913deB6f67967B99D67aCDFa1712C293601', 'transactionIndex': 18, 'v': 28, 'value': 12345})
Target balance in gweis: 2196500005635337658
target balance in eth: 2.196500005635337658
Balance in gweis: 999946843749925930
Balance in eth: 0.99994684374992593
